In [1]:
import re
import pandas as pd

#  读取 CSV
df = pd.read_csv('hotel.csv', encoding='utf-8')

#  重命名关键列
df.rename(columns={
    'タイトル':     'name',               # 宾馆名称
    'タイトルURL':  'url',                # 详情页链接
    'タイトル2':    'location',           # “石川県 > 金沢・湯涌”形式的位置字段
    '価格7':        'price_total',        # 例: “11,200円～”
    '価格8':        'price_per_person',    # 例: “1名 5,600円～”
    '概要':         'rating' 
}, inplace=True)

In [2]:
#  提取 hotel_id（URL 中的 yad123456）
def extract_hotel_id(link):
    m = re.search(r'yad\d+', str(link))
    return m.group(0) if m else pd.NA

df['hotel_id'] = df['url'].apply(extract_hotel_id)

In [3]:
#  解析 location 拆分为 prefecture / city
#    示例 "石川県 > 金沢・湯涌"
loc_split = df['location'].str.split(' > ', expand=True)
df['prefecture'] = loc_split[0]
df['city']       = loc_split[1]

In [4]:
#  价格字段清洗：去除日元符号、逗号，“～”等，提取数字
for col in ['price_total', 'price_per_person']:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r'[¥,～\s名名人大人]+', '', regex=True)  # 去掉 ¥ , ～ 及文字
        .str.extract(r'(\d+)')                               # 抽出数字
        .astype(float)
    )

In [5]:
#  丢弃关键列缺失的行
df.dropna(subset=['hotel_id','name'], inplace=True)


In [6]:
# 删除重复 hotel_id，只保留第一条
df.drop_duplicates(subset='hotel_id', keep='first', inplace=True)


In [13]:

#  重建索引
df.reset_index(drop=True, inplace=True)


In [7]:

# 只保留关键列
key_cols = [
    'hotel_id',
    'name',
    'prefecture',
    'city',
    'price_per_person',
    'rating'
]
df = df[key_cols]

#  检查前几行
print(df[['hotel_id','name','prefecture','city','price_per_person','rating']].head())

# 14. 再次检查
print(df.head())



    hotel_id                                    name prefecture   city  \
0  yad378719                        FORZA　ホテルフォルツァ金沢        石川県  金沢・湯涌   
1  yad350199  THE HOTEL SANRAKU KANAZAWA（ザ ホテル山楽 金沢）        石川県  金沢・湯涌   
2  yad304736                            ホテル・トリフィート金沢        石川県  金沢・湯涌   
3  yad388181          天然温泉 加賀の宝泉 御宿 野乃金沢（ドーミーインチェーン）        石川県  金沢・湯涌   
4  yad362769                            ホテルインターゲート金沢        石川県  金沢・湯涌   

   price_per_person  rating  
0           15600.0     4.6  
1          112000.0     4.8  
2           16250.0     4.6  
3           18450.0     4.5  
4           17400.0     4.5  
    hotel_id                                    name prefecture   city  \
0  yad378719                        FORZA　ホテルフォルツァ金沢        石川県  金沢・湯涌   
1  yad350199  THE HOTEL SANRAKU KANAZAWA（ザ ホテル山楽 金沢）        石川県  金沢・湯涌   
2  yad304736                            ホテル・トリフィート金沢        石川県  金沢・湯涌   
3  yad388181          天然温泉 加賀の宝泉 御宿 野乃金沢（ドーミーインチェーン）        石川県  金沢・湯涌   
4  ya

In [8]:

#  保存前处理结果
df.to_csv('hotel_cleaned.csv', index=False, encoding='utf-8-sig')